# Workshop PyTorch - Nivel Básico
## Clasificación de Neumonía en Rayos X usando CNNs

### Objetivo del Notebook
En este notebook aprenderás a implementar una red neuronal convolucional (CNN) para clasificar imágenes de rayos X del pecho y detectar neumonía. Este es un ejemplo práctico de cómo PyTorch facilita el desarrollo de modelos de deep learning para aplicaciones médicas.

### Prerequisitos
- Conceptos básicos de PyTorch (tensores, autograd, nn.Module)
- Conocimientos básicos de redes neuronales
- Comprensión de imágenes digitales

---

## 🔧 1. Importación de Librerías
*Tiempo estimado: 2 minutos*

En esta sección importamos todas las librerías necesarias para el proyecto, incluyendo PyTorch, torchvision, numpy, matplotlib, sklearn y tqdm.

In [ ]:
# Importamos todas las librerías necesarias para nuestro proyecto
import os
import numpy as np
import matplotlib.pyplot as plt

# PyTorch y sus módulos principales
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Herramientas para manejo de imágenes
default_seed = 42
import torchvision
from torchvision import datasets, models, transforms

# Métricas y visualización
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tqdm.auto import tqdm

# Fijar semilla para reproducibilidad
np.random.seed(default_seed)
torch.manual_seed(default_seed)

print("✅ Todas las librerías importadas correctamente")
print(f"🔥 Versión de PyTorch: {torch.__version__}")

## 💻 2. Configuración del Dispositivo de Cómputo
*Tiempo estimado: 1 minuto*

Detectamos y configuramos el dispositivo óptimo (CPU o GPU) para el entrenamiento del modelo.

In [ ]:
# Configuramos el dispositivo óptimo para el entrenamiento
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# Alternativa para Intel: DEVICE = "xpu" if torch.xpu.is_available() else "cpu"

print(f"🎯 Dispositivo seleccionado: {DEVICE}")
print(f"📊 Dispositivos CUDA disponibles: {torch.cuda.device_count()}")

# Información adicional del sistema
if DEVICE == "cuda":
    print(f"🚀 GPU activa: {torch.cuda.get_device_name(0)}")
    print(f"💾 Memoria GPU disponible: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 📁 3. Descarga y Preparación de Datos
*Tiempo estimado: 10 minutos (dependiendo de la conexión)*

Incluye instrucciones y comandos para descargar el dataset de Kaggle y preparar la estructura de carpetas.

In [ ]:
!pip install gdown

# The Google Drive file ID is extracted from the shared link.
# The shared link is: 'https://drive.google.com/file/d/1L2fXq1eabwdk10iM_eUou_W_nEdFAIjN/view?usp=sharing'
# The file ID is '1L2fXq1eabwdk10iM_eUou_W_nEdFAIjN'
file_id = '1L2fXq1eabwdk10iM_eUou_W_nEdFAIjN'
output_filename = 'chest-xray-pneumonia.zip'

# Use gdown to download the file by its ID
!gdown --id {file_id} -O {output_filename}

In [ ]:
# IMPORTANTE: Esta celda requiere configuración previa de Kaggle
# Para usar esta celda, necesitas:
# 1. Crear una cuenta en Kaggle
# 2. Descargar tu archivo kaggle.json
# 3. Descomenta las líneas siguientes para descargar el dataset:

# !pip install -q kaggle
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

# !kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip -q chest-xray-pneumonia.zip

print("⚠️ Asegúrate de tener el dataset descargado en ./chest_xray/")
print("📋 Estructura esperada:")
print("   ./chest_xray/")
print("   ├── train/")
print("   ├── val/")
print("   └── test/")

## 🗂️ 4. Definición de Rutas y Transformaciones de Datos
*Tiempo estimado: 5 minutos*

Definimos las rutas a los conjuntos de datos y aplicamos transformaciones de aumento de datos y normalización para entrenamiento, validación y prueba.

In [ ]:
# Definimos las rutas de nuestros datos
data_dir = './chest_xray/'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

# Configuración de las imágenes
target_image_size = 224  # Tamaño estándar para muchas arquitecturas pre-entrenadas

# 🎨 TRANSFORMACIONES PARA ENTRENAMIENTO
train_transforms = transforms.Compose([
    transforms.Resize((target_image_size, target_image_size)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 🎯 TRANSFORMACIONES PARA VALIDACIÓN Y PRUEBA
val_test_transforms = transforms.Compose([
    transforms.Resize((target_image_size, target_image_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("✅ Transformaciones definidas correctamente")
print(f"📏 Tamaño de imagen: {target_image_size}x{target_image_size}")
print("🔄 Aumento de datos activado para entrenamiento")

## 📊 5. Carga de Datasets con ImageFolder
*Tiempo estimado: 3 minutos*

Cargamos los datasets de imágenes usando torchvision.datasets.ImageFolder y mostramos información sobre las clases y el balance de datos.

In [ ]:
# PyTorch facilita la carga de datos con ImageFolder
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset = datasets.ImageFolder(val_dir, transform=val_test_transforms)
test_dataset = datasets.ImageFolder(test_dir, transform=val_test_transforms)

# Información del dataset
print(f"🏷️ Clases detectadas: {train_dataset.classes}")
print(f"📝 Mapeo de clases: {train_dataset.class_to_idx}")
print(f"📚 Imágenes de entrenamiento: {len(train_dataset):,}")
print(f"🔍 Imágenes de validación: {len(val_dataset):,}")
print(f"🧪 Imágenes de prueba: {len(test_dataset):,}")

# Balance de clases en entrenamiento
train_normal = len([x for x in train_dataset.imgs if x[1] == 0])
train_pneumonia = len([x for x in train_dataset.imgs if x[1] == 1])
print(f"\n📊 Balance de clases en entrenamiento:")
print(f"   🫁 Normal: {train_normal:,} ({train_normal/len(train_dataset)*100:.1f}%)")
print(f"   🦠 Neumonía: {train_pneumonia:,} ({train_pneumonia/len(train_dataset)*100:.1f}%)")

## 🚀 6. Creación de DataLoaders
*Tiempo estimado: 2 minutos*

Creamos DataLoaders para entrenamiento, validación y prueba, configurando el batch size y la optimización para GPU.

In [ ]:
# Configuramos el batch size (tamaño de lote)
batch_size = 32

# Los DataLoaders manejan la carga eficiente de datos
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f"✅ DataLoaders creados exitosamente")
print(f"📦 Tamaño de batch: {batch_size}")
print(f"🔄 Batches de entrenamiento: {len(train_loader)}")
print(f"🔍 Batches de validación: {len(val_loader)}")
print(f"🧪 Batches de prueba: {len(test_loader)}")

## 🖼️ 7. Visualización de Datos
*Tiempo estimado: 5 minutos*

Mostramos ejemplos de imágenes del dataset con sus etiquetas para verificar la correcta carga y transformación de los datos.

In [ ]:
def visualizar_imagen(tensor_imagen, titulo=None):
    """
    Función para desnormalizar y mostrar una imagen desde tensor
    """
    imagen_np = tensor_imagen.numpy().transpose((1, 2, 0))
    media = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    imagen_np = std * imagen_np + media
    imagen_np = np.clip(imagen_np, 0, 1)
    plt.imshow(imagen_np)
    if titulo:
        plt.title(titulo)
    plt.axis('off')

# Obtenemos un batch de datos de entrenamiento
batch_imagenes, batch_etiquetas = next(iter(train_loader))

# Creamos una grilla con las primeras 8 imágenes
fig, axes = plt.subplots(2, 4, figsize=(15, 8))
axes = axes.ravel()
nombres_clases = train_dataset.classes
for i in range(8):
    ax = axes[i]
    plt.sca(ax)
    visualizar_imagen(batch_imagenes[i], f"{nombres_clases[batch_etiquetas[i]]}")
plt.tight_layout()
plt.show()

print(f"🎨 Visualización de {len(batch_imagenes)} imágenes del batch")
print(f"📊 Forma del batch: {batch_imagenes.shape}")

## 🧠 8. Construcción de la Red Neuronal Convolucional
*Tiempo estimado: 10 minutos*

Definimos la arquitectura de la CNN personalizada para la clasificación binaria y mostramos el resumen del modelo y el número de parámetros entrenables.

In [ ]:
class CNN_Neumonia(nn.Module):
    """
    Red Neuronal Convolucional para clasificación de neumonía
    Arquitectura:
    - 3 capas convolucionales con ReLU y MaxPooling
    - 2 capas fully connected con Dropout
    - Salida binaria (Normal vs Neumonía)
    """
    def __init__(self):
        super(CNN_Neumonia, self).__init__()
        self.capas_conv = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.capas_fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 28 * 28, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1)
        )
    def forward(self, x):
        x = self.capas_conv(x)
        x = self.capas_fc(x)
        return x
    def contar_parametros(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

modelo = CNN_Neumonia().to(DEVICE)
print("🧠 Modelo creado exitosamente")
print(f"📊 Parámetros entrenables: {modelo.contar_parametros():,}")
print(f"💾 Dispositivo del modelo: {next(modelo.parameters()).device}")
print("\n📋 Arquitectura del modelo:")
print(modelo)

## ⚙️ 9. Configuración de Pérdida y Optimizador
*Tiempo estimado: 3 minutos*

Configuramos la función de pérdida BCEWithLogitsLoss, el optimizador Adam y un scheduler para el learning rate.

In [ ]:
# 📉 FUNCIÓN DE PÉRDIDA
funcion_perdida = nn.BCEWithLogitsLoss()

# 🚀 OPTIMIZADOR
optimizador = optim.Adam(modelo.parameters(), lr=1e-4)

# 📊 SCHEDULER (Opcional)
scheduler = optim.lr_scheduler.StepLR(optimizador, step_size=3, gamma=0.1)

print("⚙️ Configuración de entrenamiento:")
print(f"   🎯 Función de pérdida: {funcion_perdida.__class__.__name__}")
print(f"   🚀 Optimizador: {optimizador.__class__.__name__}")
print(f"   📈 Learning rate inicial: {optimizador.param_groups[0]['lr']}")
print(f"   🔄 Scheduler: {scheduler.__class__.__name__}")

## 🏋️ 10. Bucle de Entrenamiento Principal
*Tiempo estimado: 15-30 minutos dependiendo del hardware*

Implementamos el ciclo de entrenamiento y validación por épocas, almacenando el historial de métricas y mostrando el progreso.

In [ ]:
EPOCHS = 1  # Número de épocas
historial = {
    'perdida_entrenamiento': [],
    'perdida_validacion': [],
    'precision_validacion': []
}

print("🚀 Iniciando entrenamiento...")
print("=" * 60)

for epoca in range(EPOCHS):
    print(f"\n🔄 Época {epoca + 1}/{EPOCHS}")
    print("-" * 50)
    # ======== FASE DE ENTRENAMIENTO ========
    modelo.train()
    perdida_entrenamiento = 0.0
    barra_entrenamiento = tqdm(train_loader, desc="🏋️ Entrenando", leave=False)
    for batch_imagenes, batch_etiquetas in barra_entrenamiento:
        batch_imagenes = batch_imagenes.to(DEVICE)
        batch_etiquetas = batch_etiquetas.to(DEVICE).float().unsqueeze(1)
        optimizador.zero_grad()
        predicciones = modelo(batch_imagenes)
        perdida = funcion_perdida(predicciones, batch_etiquetas)
        perdida.backward()
        optimizador.step()
        perdida_entrenamiento += perdida.item() * batch_imagenes.size(0)
        barra_entrenamiento.set_postfix({'Pérdida': f"{perdida.item():.4f}"})
    # ======== FASE DE VALIDACIÓN ========
    modelo.eval()
    perdida_validacion = 0.0
    predicciones_correctas = 0
    barra_validacion = tqdm(val_loader, desc="🔍 Validando", leave=False)
    with torch.no_grad():
        for batch_imagenes, batch_etiquetas in barra_validacion:
            batch_imagenes = batch_imagenes.to(DEVICE)
            batch_etiquetas = batch_etiquetas.to(DEVICE).float().unsqueeze(1)
            predicciones = modelo(batch_imagenes)
            perdida = funcion_perdida(predicciones, batch_etiquetas)
            perdida_validacion += perdida.item() * batch_imagenes.size(0)
            predicciones_binarias = torch.sigmoid(predicciones) > 0.5
            predicciones_correctas += torch.sum(predicciones_binarias == batch_etiquetas)
    perdida_prom_entrenamiento = perdida_entrenamiento / len(train_dataset)
    perdida_prom_validacion = perdida_validacion / len(val_dataset)
    precision_validacion = predicciones_correctas.double() / len(val_dataset)
    historial['perdida_entrenamiento'].append(perdida_prom_entrenamiento)
    historial['perdida_validacion'].append(perdida_prom_validacion)
    historial['precision_validacion'].append(precision_validacion.item())
    scheduler.step()
    print(f"📊 Resultados de la época {epoca + 1}:")
    print(f"   🏋️ Pérdida entrenamiento: {perdida_prom_entrenamiento:.4f}")
    print(f"   🔍 Pérdida validación: {perdida_prom_validacion:.4f}")
    print(f"   🎯 Precisión validación: {precision_validacion:.4f} ({precision_validacion*100:.1f}%)")
    print(f"   📈 Learning rate: {optimizador.param_groups[0]['lr']:.6f}")

print("\n🎉 ¡Entrenamiento completado exitosamente!")

## 📈 11. Visualización de Curvas de Aprendizaje
*Tiempo estimado: 5 minutos*

Graficamos la evolución de la pérdida y precisión durante el entrenamiento y validación, e identificamos posibles signos de overfitting.

In [ ]:
plt.figure(figsize=(15, 5))
# Gráfico 1: Pérdida vs Épocas
plt.subplot(1, 3, 1)
plt.plot(range(1, EPOCHS + 1), historial['perdida_entrenamiento'], 'b-o', label='Entrenamiento', linewidth=2, markersize=6)
plt.plot(range(1, EPOCHS + 1), historial['perdida_validacion'], 'r-o', label='Validación', linewidth=2, markersize=6)
plt.title('📉 Evolución de la Pérdida', fontsize=14, fontweight='bold')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.grid(True, alpha=0.3)
# Gráfico 2: Precisión vs Épocas
plt.subplot(1, 3, 2)
plt.plot(range(1, EPOCHS + 1), historial['precision_validacion'], 'g-o', label='Validación', linewidth=2, markersize=6)
plt.title('🎯 Evolución de la Precisión', fontsize=14, fontweight='bold')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.legend()
plt.grid(True, alpha=0.3)
# Gráfico 3: Comparación Pérdida Entrenamiento vs Validación
plt.subplot(1, 3, 3)
diferencia = np.array(historial['perdida_validacion']) - np.array(historial['perdida_entrenamiento'])
plt.plot(range(1, EPOCHS + 1), diferencia, 'purple', linewidth=2, marker='o')
plt.title('🔍 Diferencia de Pérdida\n(Validación - Entrenamiento)', fontsize=14, fontweight='bold')
plt.xlabel('Épocas')
plt.ylabel('Diferencia')
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

# Análisis automático de las curvas
print("🔍 Análisis de las curvas de aprendizaje:")
print(f"   📊 Precisión final: {historial['precision_validacion'][-1]:.4f}")
print(f"   📈 Mejora en precisión: {historial['precision_validacion'][-1] - historial['precision_validacion'][0]:.4f}")
if historial['perdida_validacion'][-1] > historial['perdida_entrenamiento'][-1]:
    print("   ⚠️  Posible overfitting detectado")
else:
    print("   ✅ No hay signos claros de overfitting")

## 🎭 12. Matriz de Confusión y Evaluación Final
*Tiempo estimado: 5 minutos*

Evaluamos el modelo en el conjunto de prueba, generamos la matriz de confusión y calculamos métricas detalladas como precisión, sensibilidad y especificidad.

In [ ]:
print("🧪 Evaluando modelo en conjunto de prueba...")
todas_predicciones = []
todas_etiquetas = []
probabilidades = []
modelo.eval()
with torch.no_grad():
    for batch_imagenes, batch_etiquetas in tqdm(test_loader, desc="Evaluando"):
        batch_imagenes = batch_imagenes.to(DEVICE)
        batch_etiquetas = batch_etiquetas.to(DEVICE)
        logits = modelo(batch_imagenes)
        probs = torch.sigmoid(logits)
        predicciones = probs > 0.5
        todas_predicciones.extend(predicciones.cpu().numpy().flatten())
        todas_etiquetas.extend(batch_etiquetas.cpu().numpy())
        probabilidades.extend(probs.cpu().numpy().flatten())
todas_predicciones = np.array(todas_predicciones, dtype=int)
todas_etiquetas = np.array(todas_etiquetas)
probabilidades = np.array(probabilidades)
cm = confusion_matrix(todas_etiquetas, todas_predicciones)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=test_dataset.classes)
plt.figure(figsize=(8, 6))
disp.plot(cmap='Blues', values_format='d')
plt.title('🎭 Matriz de Confusión - Conjunto de Prueba', fontsize=16, fontweight='bold')
plt.show()
from sklearn.metrics import classification_report, accuracy_score
print("\n📊 Reporte de Clasificación:")
print(classification_report(todas_etiquetas, todas_predicciones, target_names=test_dataset.classes))
accuracy = accuracy_score(todas_etiquetas, todas_predicciones)
tn, fp, fn, tp = cm.ravel()
sensitivity = tp / (tp + fn)  # Sensibilidad (Recall)
specificity = tn / (tn + fp)  # Especificidad
precision = tp / (tp + fp)    # Precisión
print(f"\n🎯 Métricas Importantes:")
print(f"   📈 Precisión General: {accuracy:.4f} ({accuracy*100:.1f}%)")
print(f"   🔍 Sensibilidad (Recall): {sensitivity:.4f} ({sensitivity*100:.1f}%)")
print(f"   ✅ Especificidad: {specificity:.4f} ({specificity*100:.1f}%)")
print(f"   🎯 Precisión (Pneumonia): {precision:.4f} ({precision*100:.1f}%)")
print(f"\n🔢 Matriz de Confusión:")
print(f"   ✅ Verdaderos Negativos (TN): {tn}")
print(f"   ❌ Falsos Positivos (FP): {fp}")
print(f"   ❌ Falsos Negativos (FN): {fn}")
print(f"   ✅ Verdaderos Positivos (TP): {tp}")

## 🤔 13. Reflexión Ética y Discusión
*Tiempo estimado: 10 minutos*

Analizamos los errores del modelo desde una perspectiva médica y ética, calculamos el AUC-ROC y visualizamos la curva ROC.

In [ ]:
print("🏥 ANÁLISIS ÉTICO Y MÉDICO")
print("=" * 50)
print(f"❌ Falsos Positivos (FP = {fp}):")
print("   • Diagnosticar neumonía a una persona sana")
print("   • Consecuencias: Estrés, pruebas adicionales, costos")
print("   • Gravedad: MEDIA")
print(f"\n❌ Falsos Negativos (FN = {fn}):")
print("   • NO diagnosticar neumonía a una persona enferma")
print("   • Consecuencias: Progresión de la enfermedad, riesgo de vida")
print("   • Gravedad: ALTA")
print(f"\n🎯 En medicina, generalmente preferimos:")
print("   • ALTA sensibilidad (pocos falsos negativos)")
print("   • Aceptar más falsos positivos que falsos negativos")
print("   • Principio: 'Mejor prevenir que lamentar'")
from sklearn.metrics import roc_curve, auc
fpr, tpr, thresholds = roc_curve(todas_etiquetas, probabilidades)
roc_auc = auc(fpr, tpr)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
print(f"\n📊 Análisis ROC:")
print(f"   🎯 AUC-ROC: {roc_auc:.4f}")
print(f"   ⚖️ Punto de corte óptimo: {optimal_threshold:.4f}")
print(f"   📈 Sensibilidad óptima: {tpr[optimal_idx]:.4f}")
print(f"   📉 Especificidad óptima: {1-fpr[optimal_idx]:.4f}")
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('Tasa de Falsos Positivos (1 - Especificidad)')
plt.ylabel('Tasa de Verdaderos Positivos (Sensibilidad)')
plt.title('Curva ROC - Clasificación de Neumonía')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.show()